#### _Exercise 3.1.1:_ Determine different signing scenarios and their likelihoods

##### Spending paths

_Spending paths in order of likelihood:_

1. 3 Main wallets sign.
2. 2 Main wallets & 1 Backup wallet.
3. 1 Main wallet & 2 Backup wallet.

##### Taproot Descriptors

_Sketch out taproot descriptors:_

1. Internalkey: `MuSig(pkA, pkB, pkC)`
2. 2 main keys & 1 backup key:
  - `csa_delay(3, pkA, pkB, pkD, 3 days)`
  - `csa_delay(3, pkA, pkC, pkD, 3 days)`
  - `csa_delay(3, pkB, pkC, pkD, 3 days)`
  - `csa_delay(3, pkA, pkB, pkE, 3 days)`
  - `csa_delay(3, pkA, pkC, pkE, 3 days)`
  - `csa_delay(3, pkB, pkC, pkE, 3 days)`
3. 1 main keys & 2 backup keys:
  - `csa_delay(3, pkA, pkD, pkE, 10 days)`
  - `csa_delay(3, pkB, pkD, pkE, 10 days)`
  - `csa_delay(3, pkC, pkD, pkE, 10 days)`

**Note: since backup keys cannot participate in MuSig, all possible key combinations are enumerated in different leaves of the Taptree.

#### _Programming Exercise 3.1.3:_ Build a taproot output

In [ ]:
# Tapscripts - 2 main keys & 1 backup key
# Use construct_csa_delay() to construct the tapscript
delay = 3*24*6
tapscript_2a = TapLeaf().construct_csa_delay(3, [main_pubkeyA, main_pubkeyB, backup_pubkeyD], delay)
tapscript_2b = TapLeaf().construct_csa_delay(3, [main_pubkeyA, main_pubkeyC, backup_pubkeyD], delay)
tapscript_2c = TapLeaf().construct_csa_delay(3, [main_pubkeyB, main_pubkeyC, backup_pubkeyD], delay)
tapscript_2d = TapLeaf().construct_csa_delay(3, [main_pubkeyA, main_pubkeyB, backup_pubkeyE], delay)
tapscript_2e = TapLeaf().construct_csa_delay(3, [main_pubkeyA, main_pubkeyC, backup_pubkeyE], delay)
tapscript_2f = TapLeaf().construct_csa_delay(3, [main_pubkeyB, main_pubkeyC, backup_pubkeyE], delay)

# Tapscripts - 1 main keys & 2 backup keys
long_delay = 10*24*6
tapscript_3a = TapLeaf().construct_csa_delay(3, [main_pubkeyA, backup_pubkeyD, backup_pubkeyE], long_delay)
tapscript_3b = TapLeaf().construct_csa_delay(3, [main_pubkeyB, backup_pubkeyD, backup_pubkeyE], long_delay)
tapscript_3c = TapLeaf().construct_csa_delay(3, [main_pubkeyC, backup_pubkeyD, backup_pubkeyE], long_delay)

# Set list of backup tapscripts
# Suggestion: Include tapscripts with 3d timelocks first, then those with 10d timelocks
backup_tapscripts = [tapscript_2a, tapscript_2b, tapscript_2c,
                     tapscript_2d, tapscript_2e, tapscript_2f,
                     tapscript_3a, tapscript_3b, tapscript_3c]

assert len(backup_tapscripts) == 9

# Construct taptree with huffman constructor
tapscript_weights = [(2, tapscript_2a), (2, tapscript_2b), (2, tapscript_2c),
                     (2, tapscript_2d), (2, tapscript_2e), (2, tapscript_2f),
                     (1, tapscript_3a), (1, tapscript_3b), (2, tapscript_3c)]
multisig_taproot = TapTree(key=musig_ABC)
multisig_taproot.huffman_constructor(tapscript_weights)

print("Taproot descriptor: {}\n".format(multisig_taproot.desc))

# Derive segwit v1 address
tapscript, taptweak, control_map = multisig_taproot.construct()
taptweak = int.from_bytes(taptweak, 'big')
output_pubkey = musig_ABC.tweak_add(taptweak)
output_pubkey_b = output_pubkey.get_bytes()
segwit_address = program_to_witness(1, output_pubkey_b)
print("Segwit Address:", segwit_address)

#### 3.1.4 _Programming Exercise:_ Create a valid key path output

In [ ]:
# Negate keys if necessary
output_keyPath = output_pubkey
privKeyA_keyPath = main_privkeyA_c
privKeyB_keyPath = main_privkeyB_c
privKeyC_keyPath = main_privkeyC_c
tweak_keyPath = taptweak

if output_keyPath.get_y()%2 != 0:
    output_keyPath.negate()
    privKeyA_keyPath.negate()
    privKeyB_keyPath.negate()
    privKeyC_keyPath.negate()
    tweak_keyPath = SECP256K1_ORDER - taptweak

# Create sighash for ALL
sighash_musig = TaprootSignatureHash(spending_tx, [output], SIGHASH_ALL_TAPROOT)
 
# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
nonceA = generate_schnorr_nonce()
nonceB = generate_schnorr_nonce()
nonceC = generate_schnorr_nonce()
R_agg, negated = aggregate_schnorr_nonces([nonceA.get_pubkey(), nonceB.get_pubkey(), nonceC.get_pubkey()])
if negated:
    nonceA.negate()
    nonceB.negate()
    nonceC.negate()

# Create an aggregate signature.
# Remember to add a factor for the tweak
sA = sign_musig(privKeyA_keyPath, nonceA, R_agg, output_pubkey, sighash_musig)
sB = sign_musig(privKeyB_keyPath, nonceB, R_agg, output_pubkey, sighash_musig)
sC = sign_musig(privKeyC_keyPath, nonceC, R_agg, output_pubkey, sighash_musig)
e = musig_digest(R_agg, output_keyPath, sighash_musig)
sig_agg = aggregate_musig_signatures([sA, sB, sC, e * tweak_keyPath], R_agg)

print("Aggregate signature is {}\n".format(sig_agg.hex()))

assert output_keyPath.verify_schnorr(sig_agg, sighash_musig)

# Construct transaction witness
spending_tx.wit.vtxinwit.append(CTxInWitness([sig_agg]))
 
print("spending_tx: {}\n".format(spending_tx))

# Test mempool acceptance
spending_tx_str = spending_tx.serialize().hex() 
assert test.nodes[0].testmempoolaccept([spending_tx_str])[0]['allowed']

print("Key path spending transaction weight: {}".format(test.nodes[0].decoderawtransaction(spending_tx_str)['weight']))

print("Success!")

#### 3.1.6 _Programming Exercise:_ Create a valid script path output for a long delay script

In [ ]:
# Construct transaction
spending_tx = CTransaction()

spending_tx.nVersion = 2
spending_tx.nLockTime = 0
outpoint = COutPoint(tx.sha256, output_index)
spending_tx_in = CTxIn(outpoint=outpoint, nSequence=long_delay)
spending_tx.vin = [spending_tx_in]
spending_tx.vout = [dest_output]

# Derive the sighash. Use tapscript_3a.
sighash = TaprootSignatureHash(spending_tx, [output], SIGHASH_ALL_TAPROOT, 0, scriptpath=True, script=tapscript_3a.script)

witness_elements = []

# Add signatures to the witness
# Remember to reverse the order of signatures
sigA = main_privkeyA.sign_schnorr(sighash)
sigD = backup_privkeyD.sign_schnorr(sighash)
sigE = backup_privkeyE.sign_schnorr(sighash)

# Construct transaction witness
witness_elements = [sigE, sigD, sigA, tapscript_3a.script, control_map[tapscript_3a.script]]
spending_tx.wit.vtxinwit.append(CTxInWitness(witness_elements))
spending_tx_str = spending_tx.serialize().hex()

# Test timelock
assert_equal(
    [{'txid': spending_tx.rehash(), 'allowed': False, 'reject-reason': 'non-BIP68-final'}],
    test.nodes[0].testmempoolaccept([spending_tx_str])
)

print("Long delay script path spending transaction weight: {}".format(test.nodes[0].decoderawtransaction(spending_tx_str)['weight']))

print("Success!")